# Ollama + RAG + LangChain Demo

---
## 1. Imports und Setup
Hier werden alle benötigten Bibliotheken importiert.


In [65]:
from langchain_core.runnables import RunnableConfig
from langchain_ollama import ChatOllama
from langchain_core.documents import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.chains import RetrievalQA  # aktualisierter Import
from langchain_community.document_loaders.mongodb import MongodbLoader
from langchain_community.document_loaders.pdf import PyPDFLoader
# Optional: Direktzugriff auf MongoDB, falls benötigt
from pymongo import MongoClient
from dotenv import load_dotenv


## 6. LangSmith-Anbindung

In [66]:
# .env laden und LangSmith-Umgebungsvariablen setzen
from dotenv import load_dotenv
import os

# Lade .env (funktioniert auch im Jupyter Notebook, wenn die Datei im Projekt liegt)
load_dotenv(dotenv_path="./.env")

# Setze explizit die LangSmith-Variablen für LangChain
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGSMITH_TRACING", "true")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT", "")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY", "")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGSMITH_PROJECT", "")

# Check, ob die wichtigsten Variablen korrekt geladen wurden
print("LangSmith-Konfiguration:")
print("LANGCHAIN_TRACING_V2:", os.environ.get("LANGCHAIN_TRACING_V2"))
print("LANGCHAIN_ENDPOINT:", os.environ.get("LANGCHAIN_ENDPOINT"))
print("LANGCHAIN_API_KEY gesetzt:", bool(os.environ.get("LANGCHAIN_API_KEY")))
print("LANGCHAIN_PROJECT:", os.environ.get("LANGCHAIN_PROJECT"))


LangSmith-Konfiguration:
LANGCHAIN_TRACING_V2: true
LANGCHAIN_ENDPOINT: https://api.smith.langchain.com
LANGCHAIN_API_KEY gesetzt: True
LANGCHAIN_PROJECT: Quickstart: LangGraphJS


## 2. LLM-Instanz für Ollama (Mistral)

In [73]:
system_prompt = f"""
You are a helpful assistant for store employees in a pet supply shop.

Your main role is to support them in fulfilling Click & Collect orders. This includes:
- Understanding and interpreting order details,
- Suggesting relevant additional products (e.g. food, accessories, care items),
- Preparing customer dialogues for handover (e.g. greeting, product tips, upselling).

Always respond in the language the user used in their message. If the user changes the language, adapt accordingly. Be clear, concise, and helpful.

Be friendly, competent, and focused on customer satisfaction.

The User asks you:
"""

llm = ChatOllama(
    model="mistral",
    temperature=0.3,
    top_p=0.3,

)


## 3. Hilfsfunktionen für Dokumentverarbeitung und Vektorstore

In [68]:
def split_documents(docs, chunk_size=100, chunk_overlap=10):
    splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(docs)

def build_vectorstore(docs, embeddings_model="mistral", persist_directory=None):
    embeddings = OllamaEmbeddings(model=embeddings_model)
    if persist_directory:
        return Chroma.from_documents(docs, embedding=embeddings, persist_directory=persist_directory)
    else:
        return Chroma.from_documents(docs, embedding=embeddings)

def build_qa_chain(llm, retriever):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )


## 4. PDF-Dokument laden und QA-Chain aufbauen

In [69]:
from langchain_community.document_loaders.pdf import PyPDFLoader

def setup_pdf_chain(pdf_path, llm):
    loader = PyPDFLoader(pdf_path)
    pdf_docs = loader.load()
    split_pdf_docs = split_documents(pdf_docs)
    if not split_pdf_docs:
        print("Warnung: Keine PDF-Dokumente zum Indexieren gefunden.")
        return None, None, None
    vectorstore_pdf = build_vectorstore(split_pdf_docs)
    retriever_pdf = vectorstore_pdf.as_retriever()
    qa_chain_pdf = build_qa_chain(llm, retriever_pdf)
    return qa_chain_pdf, retriever_pdf, vectorstore_pdf

pdf_path = "ressources/pdf/Arbeitsanweisung_Tierbedarfsladen.pdf"
qa_chain_pdf, retriever_pdf, vectorstore_pdf = setup_pdf_chain(pdf_path, llm)


## 5. MongoDB-Dokumente laden und QA-Chain aufbauen

In [ ]:
def load_mongo_docs():
    client = MongoClient("mongodb://localhost:27017/")
    collection = client["thesis"]["tasks"]
    raw_docs = list(collection.find())

    def format_task(doc):
        products_text = "\n".join([f"- {p['quantity']}x {p['sku']}" for p in doc.get("products", [])])
        return f"""
Aufgabe {doc.get("taskId", "Unbekannt")} für {doc.get("customerName", "Unbekannt")} ({doc.get("customerId")}):
Status: {doc.get("status")}
Zeitraum: {doc.get("startDate")} bis {doc.get("endDate")}
Warnhinweis: {"ja" if doc.get("hasWarning") else "nein"}
Produkte:
{products_text}
""".strip()

    mongo_docs = [
        Document(
            page_content=format_task(doc),
            metadata={"_id": str(doc["_id"])}
        )
        for doc in raw_docs
    ]
    return mongo_docs

def setup_mongo_chain(llm):
    mongo_docs = load_mongo_docs()
    split_docs = split_documents(mongo_docs, chunk_size=500, chunk_overlap=50)
    if not split_docs:
        print("Keine Dokumente zum Indexieren gefunden.")
        return None, None, None
    vectorstore = build_vectorstore(split_docs, persist_directory="./chroma_mongo")
    retriever = vectorstore.as_retriever()
    qa_chain = build_qa_chain(llm, retriever)
    return qa_chain, retriever, vectorstore

qa_chain_mongo, retriever_mongo, vectorstore_mongo = setup_mongo_chain(llm)


## 6. Agent mit Tools für PDF- und MongoDB-QA

In [ ]:
from langchain.agents import Tool, initialize_agent, AgentType

tools = []
if qa_chain_pdf:
    tools.append(
        Tool(
            name="PDF QA",
            func=lambda input: qa_chain_pdf.invoke({"query": input})["result"],
            description="Beantworte Fragen zu den Inhalten der PDF-Arbeitsanweisung."
        )
    )
if qa_chain_mongo:
    tools.append(
        Tool(
            name="MongoDB QA",
            func=lambda input: qa_chain_mongo.invoke({"query": input})["result"],
            description="Beantworte Fragen zu Aufgaben aus der MongoDB."
        )
    )

if tools:
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
    )
else:
    agent = None
    print("Keine Tools für den Agenten verfügbar.")


## 7. Beispiel-Query an den Agenten

In [ ]:
if agent:
    query = "Welche Aufgabe ist die neueste? Antworte auf Deutsch."
    print("Agent-Antwort:", agent.invoke(query))
else:
    print("Agent konnte nicht initialisiert werden.")


In [ ]:
# Schritt 1: Trainingsdaten für LoRa-Tuning vorbereiten
import json

train_data = [
    {
        "instruction": "Was ist Click & Collect?",
        "input": "",
        "output": "Click & Collect bedeutet, dass Kunden online bestellen und die Ware im Laden abholen."
    },
    {
        "instruction": "Nenne drei Vorteile von Click & Collect.",
        "input": "",
        "output": "1. Schnelle Abholung\n2. Keine Versandkosten\n3. Persönliche Beratung im Laden"
    }
    # ...weitere Beispiele ergänzen...
]

with open("lora-train-data.jsonl", "w", encoding="utf-8") as f:
    for entry in train_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")
print("Trainingsdaten gespeichert: lora-train-data.jsonl")


In [ ]:
# Schritt 2: LoRa-Tuning mit Ollama CLI (im Terminal ausführen, nicht im Notebook)
# !ollama tune mistral lora-train-data.jsonl --output mistral-lora


In [ ]:
# Schritt 3: Das getunte Modell laden
from langchain_ollama import ChatOllama

llm_lora = ChatOllama(
    model="mistral-lora",
    temperature=0.3,
    top_p=0.3,
)


In [ ]:
# Schritt 4: Vergleich Basismodell vs. LoRa-Modell
test_prompt = "Was ist Click & Collect?"

print("Antwort Basismodell:")
print(llm.invoke(test_prompt))

print("\nAntwort LoRa-getuntes Modell:")
print(llm_lora.invoke(test_prompt))


In [ ]:
# Schritt 5: Chains und Agenten mit LoRa-Modell testen
qa_chain_pdf_lora, retriever_pdf_lora, vectorstore_pdf_lora = setup_pdf_chain(pdf_path, llm_lora)
qa_chain_mongo_lora, retriever_mongo_lora, vectorstore_mongo_lora = setup_mongo_chain(llm_lora)

tools_lora = []
if qa_chain_pdf_lora:
    tools_lora.append(
        Tool(
            name="PDF QA (LoRa)",
            func=lambda input: qa_chain_pdf_lora.invoke({"query": input})["result"],
            description="Beantworte Fragen zu den Inhalten der PDF-Arbeitsanweisung (LoRa-getuntes Modell)."
        )
    )
if qa_chain_mongo_lora:
    tools_lora.append(
        Tool(
            name="MongoDB QA (LoRa)",
            func=lambda input: qa_chain_mongo_lora.invoke({"query": input})["result"],
            description="Beantworte Fragen zu Aufgaben aus der MongoDB (LoRa-getuntes Modell)."
        )
    )

if tools_lora:
    agent_lora = initialize_agent(
        tools=tools_lora,
        llm=llm_lora,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
    )
    print("LoRa-Agent bereit.")
else:
    agent_lora = None
    print("Keine Tools für den LoRa-Agenten verfügbar.")


In [ ]:
# Beispiel-Query an den LoRa-Agenten
if agent_lora:
    query = "Welche Aufgabe ist die neueste? Antworte auf Deutsch."
    print("LoRa-Agent-Antwort:", agent_lora.invoke(query))
else:
    print("LoRa-Agent konnte nicht initialisiert werden.")
